# Assignment 1

## Part 1 

### part 1.1

First I'd like to start with importing all the modules i'm going to use in this assignment:

In [1]:
import nltk
import os
import re, pprint, collections
from urllib import request
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams
import numpy as np
from os.path import abspath, dirname, join
import inspect
import matplotlib.pyplot as plt
import matplotlib
from math import log
import random
from collections import *
import zipfile
import string

We use the following code in order to tokenize the data based on the ptb method:

In [ ]:
def is_special_char(x):
    return ((x=='N') or ('\n' in x))

def replace_numbers(tokens):
    return [x if not x.isdigit() else 'N' for x in tokens]

def find_most_common(tokens, top):
    r = re.compile("^[a-zA-Z]+$")
    word_tokens = list(filter(r.match, tokens))
    counter = collections.Counter(word_tokens)
    most_common = counter.most_common(top)
    return [a for a, b in most_common]

def replace_noncommon_tokens(tokens, most_common):
    return [x if ((x in most_common) or (is_special_char(x))) else '<unk>' for x in tokens]

def ptb_preprocess(filenames, top=10000):
    for single_file in filenames:
    path = nltk.data.find(single_file)
    raw = open(path, 'r').read()
    
    # clean punctuations
    tokenizer = RegexpTokenizer('\w+|\$|\#|\@|\%|\&|\*|\^|\~|\<|\>|\=|\+|\n')
    tokens = tokenizer.tokenize(raw)

    # to lowercase
    words = [w.lower() for w in tokens]

    # filter numbers
    words_no_numbers = replace_numbers(words)

    # get most common words and replace all other words with unk
    common_tokens = find_most_common(words, top)
    words = replace_noncommon_tokens(words_no_numbers, common_tokens)

    # write out the new data into a file 
    new_filename = single_file + ".out"
    with open(new_filename, 'w') as f:
        for word in words:
            if ('\n' in word):
                f.write("\n")
            else:
                f.write("%s " % word)

We want to test the above code on a few example files:

In [ ]:
path_to_first_file = abspath(join(dirname("__file__"), "segmentationExample.txt"))
path_to_second_file = abspath(join(dirname("__file__"), "SplitAndPuncExample.txt"))
path_to_third_file = abspath(join(dirname("__file__"), "numbersExample.txt"))

ptb_preprocess([path_to_first_file, path_to_second_file, path_to_third_file], 10000)

For example the file "segmentationExample.text" which contains the following lines:

In [ ]:
with open(path_to_first_file) as f:
    print(f.read())

Will result in the file "segmentationExample.text.out":

In [ ]:
path_to_first_result = abspath(join(dirname("__file__"), "segmentationExample.txt.out"))
with open(path_to_first_result) as f:
    print(f.read())

The file "SplitAndPuncExample.txt" :

In [ ]:
with open(path_to_second_file) as f:
    print(f.read())

Will result in the files "SplitAndPuncExample.txt.out"

In [ ]:
path_to_second_result = abspath(join(dirname("__file__"), "SplitAndPuncExample.txt.out"))
with open(path_to_second_result) as f:
    print(f.read())

The file "numbersExample.txt":

In [ ]:
with open(path_to_third_file) as f:
    print(f.read())

Will result in the files "numbersExample.txt.out":

In [ ]:
path_to_third_result = abspath(join(dirname("__file__"), "numbersExample.txt.out"))
with open(path_to_third_result) as f:
    print(f.read())

#### part 1.1.2 

The following will plot a graph of the results of the tokenizer:

## part 1.2

the following is an implementation of the ngram model:

In [17]:
class NgramModel(object):
    def __init__(self, n, train, estimator=None):
        self._n = n
        self.is_unigram_model = (n == 1)
        cfd = nltk.ConditionalFreqDist((" ".join(train[i : i + n - 1]), "".join(train[i + n - 1])) for i in range(len(train) - n + 1))
        self._probdist = nltk.ConditionalProbDist(cfd, estimator)
        
        self._ngramsData = ngrams(train, n)
        self._ngrams = set()
        for ngram in self._ngramsData:
            self._ngrams.add(ngram)
        
        if not self.is_unigram_model:
            self._backoff = NgramModel(n - 1, train, estimator)
            self._lambda = 1
    
    def prob(self, word, context):
        if (tuple(context.split()) + (word, ) in self._ngrams) or (self.is_unigram_model):
            return self._probdist[context].prob(word)
        else:
            new_context = " ".join(context.split()[1:])
            backoff = self._backoff.prob(word, new_context)
            return self._lambda * backoff
        
    def logprob(self, word, context):
        return -(log(self.prob(word, context), 2))
    
    def get_seed(self):
        return random.choice(self._probdist.conditions())
    
    def generate(self, seed, length):
#         seed = seed.split()
#         res = list(seed)
#         for i in range(length):
#             res.append(self.generate_one(res))
#         return res
        out = []
        curr = seed
        end = self._probdist.conditions()[-1]
        i = 0
        while (i <= length and (not curr == end)):
          i += 1
          word = self._probdist[curr].generate()
          curr = " ".join((curr.split())[1:] + [word])
          out.append(word)
        return out
    
#     def generate_one(self, context):
#         keys = set(self._probdist.conditions())
#         if " ".join(context) in keys:
#             return self._probdist[context].generate()
#         elif self._n > 1:
#             return self._backoff.generate_one(context[:1])
#         else:
#             return " "

In [3]:
def train_word_lm(dataset, n=2):
    model = NgramModel(n, dataset, estimator=nltk.MLEProbDist)
    return model

the data structure used to build the model is a dictionary of size at most , but one can note we also had to save the backoff models - that is in order to do smoothing and backoff over the perplexity calculations. 

As we can see in the given class implementation - the model should export methods for evaluating itself, for generating random text, and for calculating the probabilty and entropy of a word given a context. 

## part 1.3

We'd like to measure how well our model is doing, we can do so by using a measure called perplexity - a model perplexity can be evaluated as followed:

In [4]:
def model_entropy(model, text, n=2):
    H = 0.0
    for i in range(n - 1, len(text)):
        context, word = tuple(text[i - n + 1:i]), text[i]
        context = " ".join(context)
        H += model.logprob(word, context)
    return H / float(len(text) - (n - 1))

def calc_preplexity(model, text, n=2):
    text_entropy = model_entropy(model, text, n)
    return 2 ** (text_entropy)

When testing the above implemented model on the ptb training and validation data we get:

In [6]:
ptb_train_path = abspath(join(dirname("__file__"), "ptb.train.txt"))
ptb_test_path = abspath(join(dirname("__file__"), "ptb.test.txt"))
ptb_train_tokenized = (open(ptb_train_path, 'r').read()).split()
ptb_test_tokenized = (open(ptb_test_path, 'r').read()).split()

In [ ]:
# Train the ngram model withn = 4
n = 3
lm_MLE = train_word_lm(ptb_train_tokenized, n)

print(calc_preplexity(lm_MLE, ptb_test_tokenized, n))

As mentioned, we can always use a different estimator in order to change our model perplexity, for example, the following model is using the Lidstone estimator with a gamma instead of the MLE one:

In [5]:
def train_word_lm_lidstone(dataset, n=2, gamma=0.01):
    lidstone_estimator = lambda fd: nltk.LidstoneProbDist(fd, gamma, fd.B() + 1)
    model = NgramModel(n, dataset, estimator=lidstone_estimator)
    return model

And we get the following perplexity:

In [ ]:
n = 3
gamma = 0.01
lm_LIDSTONE = train_word_lm_lidstone(ptb_train_tokenized, n, gamma)
print(calc_preplexity(lm_LIDSTONE, ptb_test_tokenized, n))

The following graph shows how differnt gamma values in such model change the results of the function:

In [ ]:
gammas = np.linspace(0, 1, 20)
n = 3
perplexities_l = list(range(20))
i = 0
for gamma in gammas:
    lm_LIDSTONE = train_word_lm_lidstone(ptb_train_tokenized, n, gamma)
    perplexities_l[i] = calc_preplexity(lm_LIDSTONE, ptb_test_tokenized, n)
    print(perplexities_l[i])
    i += 1

In [ ]:
plt.plot(gammas, perplexities_l)
plt.axis([0, 1, 100, 120])
plt.ylabel("perplexity")
plt.show()

another way is to change the value of the n-grams , and that results in different perplexity - an example of the difference can be seen in the following graph:

In [ ]:
perplexities_m = list(range(2, 20))
gamma = 0.01

for n in range(2, 20):
    lm_LIDSTONE = train_word_lm_lidstone(ptb_train_tokenized, n, gamma)    
    perp = calc_preplexity(lm_LIDSTONE, ptb_test_tokenized, n)
    print(perp)
    perplexities_m[n - 2] = perp

In [ ]:
n_vals = list(range(2, 20))

plt.plot(n_vals, perplexities_m)
plt.axis([2, 20, 1, 20])
plt.ylabel("perplexity")
plt.show()

One can see starting with a low value of n we get really high perplexity(bad) and as we increase the value the perplexity gets better. One can also note that at around n=7 the perplexity is 1, that case isn't perfect because it means the probabilities are always 0. 
So the ideal size for n is at about n=6. 

based on the two models above and the two graphs, we can now compose the "ideal" model:

In [18]:
n = 3
gamma = 0.01
lm_IDEAL = train_word_lm_lidstone(ptb_train_tokenized, n, gamma)
print(calc_preplexity(lm_IDEAL, ptb_test_tokenized, n))

107.40517847274688


compared to the results from <TODO> 

### part 1.3.2 

Another way of using the model is by generating text using it, the following method generates text given a model and a seed(a starting prefix):

In [25]:
def generate(model, seed):
    out = model.generate(seed, 100)
    out = seed + " " + " ".join(out)

    print(out)

One can note this method is not ideal if the seed length is smaller than the ngram size we used to train the model, a way to avoid such problem is to find a matching ngram starting with the seed and running the method from there (did not implemnet it) or just making sure the seed is big enough. Another way is the issue of halting. if the model run into an unknown history this code will break, so one must make sure no unknown history will occure - to do so we have the condition in the while loop. I made this generator limited to 100 iterations if possible because I didn't want to make it generate to big of a file given a large model, but one can easily change that number to anything.  

Here are a few different examples of using the previously trained model with different seeds:

In [26]:
seed = lm_IDEAL.get_seed()
print("SEED:", seed)
generate(lm_IDEAL, seed)

SEED: granted a
granted a major crop in europe that gives futures traders said the tendered units will continue making its <unk> support conservatives should take a top post at american <unk> owns five stations including several cbs inc. and oakes <unk> & <unk> a fashion magazine <unk> publications inc. owns random house four years running i hate to lose ground against other major japanese retail stores primarily in discount <unk> everything 's a lot more done if he agrees mr. kemp broke the cease-fire if the <unk> baskets too i care very much said mr. mcalpine could be one reason i began looking at just


In [27]:
seed = lm_IDEAL.get_seed()
print("SEED:", seed)
generate(lm_IDEAL, seed)

SEED: calls the
calls the <unk> funds currently offer yields almost N million offer mr. <unk> denied that the court now he says is as important as his own projections contributed to volatility he says i tried the nagging memory of one of england was a light N million of the area but has left holders of telerate inc. common stock if it uses its own <unk> harris <unk> and trust departments but <unk> are unconstitutional president bush have suggested ways of washington is as usual i 'm not sure what will come from independent producers which unlike utilities are a guide to general levels but


In [28]:
seed = lm_IDEAL.get_seed()
print("SEED:", seed)
generate(lm_IDEAL, seed)

SEED: play mr.
play mr. short 's ninth move a key member of the same time spending on the big board thursday under <unk> from about N passengers would expand a president would have veto rights to the profitable u.s. market analysts said the visit was unrelated to jaguar gm chairman roger stone said that the committee chairman henry gonzalez d. texas the lead underwriter to the farmer can say world-wide name said <unk> robert norton some fruit <unk> and <unk> <unk> drugs posed to the <unk> <unk> and <unk> <unk> has made a $ N million common shares the company faces significant challenges in some


In [29]:
seed = lm_IDEAL.get_seed()
print("SEED:", seed)
generate(lm_IDEAL, seed)

SEED: smoking patterns
smoking patterns a higher return and was unable to fend off further advances and growth allows users to believe that the latest period twice as many as one million deaths the vaccine its effectiveness before considering mandatory pro bono work we do n't think that washington 's decisions do not recoup from the spring <unk> using the new holding company the battery plant which makes hair and skin care concern at continental have n't seen the profit <unk> u s west 's u s west 's leaders in the region fighting illegal drugs in the third quarter time on the sale of the


In [ ]:
seed = random.choice(lm_IDEAL.conditions())
print("SEED:", seed)
generate(lm_IDEAL, seed)

As mentioned, this is a deterministic method that will always result in the same (or very similar in case of an equal probabilty) results. In order to create a more random generator the Temprature parameter was introduced to the generator. The idea behind temprature sampling is that the temprature allows you to give different "weights" to the probabilty - in higher temprature a more <TODO> while in a lower probabilty we get towards <TODO> .
In the generator code one can see how the equations for temprature sampling are enconded into the "sample_word" function. Firstly - we can note that 'lp/self.temp' is  exactly <TODO>

## part 1.4

The following is a summary of the first article

<TODO>

And of the second article:

<TODO>

We'd like to use Yoav Goldberg's n-gram model code:

In [ ]:
def train_char_lm(fname, order=4):
    data = open(fname, 'r').read()
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm


And train it on the cooking recipes.
First we need to gather all the recipes :

In [ ]:
# !wget http://www.ffts.com/recipes/lg/lg32965.zip

with zipfile.ZipFile("lg32965.zip","r") as zip_ref:
    zip_ref.extractall("recipes")
file_list = os.listdir("recipes")
recipes = ""
enc = 'iso-8859-15'
for file in file_list:
    recipes_from_file = open("recipes/" + file, 'r', encoding=enc).read()
    recipes_from_file.translate(string.punctuation)
    recipes += recipes_from_file

print("Number of Recipes:", recipes.count('MMMMM----- Recipe via Meal-Master (tm) v8.05'))
tokenized_recipes = word_tokenize(recipes)
num_of_tokens = len(tokenized_recipes)
print("Number of Tokens:", num_of_tokens)
vocab = sorted(set(tokenized_recipes))
print("Vocabulary size:", len(vocab))

print("Number of Chars:")

print("Distribution of the size of recipes in words and in chars:")

print("Distribution of length of words")


And then split the data into a training set, dev set and a test set:

In [ ]:
eighty_per = round(0.8 * num_of_tokens)
ten_per = round(num_of_tokens * 0.1)
with open("recipes/training.txt",'w') as f:
    f.write(" ".join(tokenized_recipes[:eighty_per]))
    f.close()
with open("recipes/dev.txt",'w') as f:
    f.write(" ".join(tokenized_recipes[eighty_per: eighty_per + ten_per]))
    f.close()
    
with open("recipes/testing.txt",'w') as f:
    f.write(" ".join(tokenized_recipes[eighty_per + ten_per: ]))
    f.close()


now we can use Yoav Goldberg's n-gram model code:

In [ ]:
order = 4
recipe_model = train_char_lm("recipes/training.txt", order)

We need to modify our perplexity method to fit a character model:

In [ ]:
def char_model_entropy(model, text, n=2):
    H = 0.0
    processed_ws = 0 
    for i in range(n - 1, len(text)):
        context, word = tuple(text[i - n:i]), text[i]
        context = "".join(context)
        score = 0
        if(context in model):
            for c,v in model[context]:
                if c == word:
                    score = v
        print(score)
        if(not(score == 0 or score == 0.0)) :
            processed_ws += 1
            H += np.log2(score)
    return - (H / float(len(text) - n))

def calc_preplexity_char(model, text, n=2):
    text_entropy = char_model_entropy(model, text, n)
    return 2 ** (text_entropy)

And the perplexity of such model is:

In [ ]:
test_data = open("recipes/testing.txt", 'r').read()
test_data.translate(string.punctuation)
pad = "~" * order
test_data = pad + test_data
tokenized_test = list(test_data)
# print(tokenized_test[0])
print(calc_preplexity_char(recipe_model, tokenized_test, order))

We also want to sample a few recipes from that model, so we will use Yoav's samling methods:

In [ ]:
def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random.random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c
            
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [ ]:
print(generate_text(recipe_model, order, 1000))

In [ ]:
print(generate_text(recipe_model, order, 1000))

In [ ]:
print(generate_text(recipe_model, order, 1000))

In [ ]:
print(generate_text(recipe_model, order, 1000))

In [ ]:
print(generate_text(recipe_model, order, 1000))